# Covid-19 (Coronavirus): Data Extract, Transform, Load (ETL) Notebook
This notebook pulls data from various sources, performs data wrangling to get it into more usable formats, and exports to CSV for use in other notebooks, as well as for future reference.  

Feeds into the Covid19_Analytics notebook.

## Data Sources

### Primary Data Source: Johns Hopkins CSSE Data Repository 
- Link: https://github.com/CSSEGISandData/COVID-19  
- **Live data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US County (cases, deaths)
- **Historic time series data:**
    - Countries (cases, deaths, recoveries, active)
    - US States (cases, deaths)
    - US Counties (cases, deaths)

### US State Testing and Hospitalizations: Covid Tracking Project 
- Link: https://covidtracking.com/data/ 
- **Live data:**
    - US State testing and hospitalization, ICU stats
- **Historic time series data:**
    - US State testing and hospitalization, ICU stats

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from datetime import datetime
import os

## Get the Datasets

### Dataset 1: John Hopkins

Live data is stored in files named MM-DD-YYYY.csv, so get the current date for live data updates.  
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_daily_reports

#### 1A: Pull the Data from JH's github repo, immediately export as 'orig'

In [2]:
# Create the filename to load based on today's date

day = str(datetime.now().day) if datetime.now().day >= 10 else "0" + str(datetime.now().day)
month = str(datetime.now().month) if datetime.now().month >= 10 else "0" + str(datetime.now().month)
year = str(datetime.now().year)

jh_live_date = month + "-" + day + "-" + year
jh_live_date

'10-20-2020'

In [3]:
#  First try to pull today's data, but if it's not available yet, pull yesterday's data
try:
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
except:
    day = str(datetime.now().day-1) if datetime.now().day-1 >= 10 else "0" + str(datetime.now().day-1)
    jh_live_date = month + "-" + day + "-" + year
    fileURL = "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_daily_reports/" + jh_live_date + ".csv"
    jh_live_global = pd.read_csv(fileURL)
    
jh_live_date

'10-20-2020'

In [4]:
# Check out the first x rows to make sure the data loaded correctly
jh_live_global.head()

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
0,NaN,NaN,NaN,Afghanistan,2020-10-21 04:24:14,33.93911,67.709953,40357,1499,33790,5068.0,Afghanistan,103.669971,3.714349
1,NaN,NaN,NaN,Albania,2020-10-21 04:24:14,41.15330,20.168300,17651,458,10225,6968.0,Albania,613.350476,2.594754
2,NaN,NaN,NaN,Algeria,2020-10-21 04:24:14,28.03390,1.659600,54829,1873,38346,14610.0,Algeria,125.034654,3.416075
3,NaN,NaN,NaN,Andorra,2020-10-21 04:24:14,42.50630,1.521800,3623,62,2273,1288.0,Andorra,4689.057141,1.711289
4,NaN,NaN,NaN,Angola,2020-10-21 04:24:14,-11.20270,17.873900,8049,251,3037,4761.0,Angola,24.490155,3.118400


Now immediately export the original jh dataset before making any changes

In [5]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_live_global_orig.csv')
jh_live_global.to_csv(filename, index=False)

**Also pull the global historic time series data.  Note that it won't have live data from today until close to midnight PT.**

In [6]:
jh_hist_global_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
jh_hist_global_cases.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,39799,39870,39928,39994,40026,40073,40141,40200,40287,40357
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,15399,15570,15752,15955,16212,16501,16774,17055,17350,17651
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,53072,53325,53399,53584,53777,53998,54203,54402,54616,54829


In [7]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_cases_orig.csv')
jh_hist_global_cases.to_csv(filename, index=False)

In [8]:
jh_hist_global_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv')
jh_hist_global_deaths.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,1477,1479,1480,1481,1481,1485,1488,1492,1497,1499
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,420,424,429,434,439,443,448,451,454,458
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,1801,1809,1818,1827,1827,1841,1846,1856,1865,1873


In [9]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_deaths_orig.csv')
jh_hist_global_deaths.to_csv(filename, index=False)

In [10]:
jh_hist_global_recovered = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_recovered_global.csv')
jh_hist_global_recovered.head(3)

,Province/State,Country/Region,Lat,Long,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,...,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20
0,NaN,Afghanistan,33.93911,67.709953,0,0,0,0,0,0,...,33114,33118,33308,33354,33447,33516,33561,33614,33760,33790
1,NaN,Albania,41.15330,20.168300,0,0,0,0,0,0,...,9500,9585,9675,9762,9864,9957,10001,10071,10167,10225
2,NaN,Algeria,28.03390,1.659600,0,0,0,0,0,0,...,37170,37382,37492,37603,37603,37856,37971,38088,38215,38346


In [11]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_global_recovered_orig.csv')
jh_hist_global_recovered.to_csv(filename, index=False)

In [12]:
jh_hist_uscounties_cases = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_US.csv')
jh_hist_uscounties_cases.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1905,1911,1924,1928,1949,1966,1983,1989,1999,2010
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,6203,6220,6248,6270,6285,6333,6350,6369,6375,6405
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,942,944,951,950,965,968,977,981,981,988


In [13]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_cases_orig.csv')
jh_hist_uscounties_cases.to_csv(filename, index=False)

In [14]:
jh_hist_uscounties_deaths = pd.read_csv('https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_US.csv')
jh_hist_uscounties_deaths.head(3)

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,28,28,28,28,28,28,28,28,28,29
1,84001003,US,USA,840,1003.0,Baldwin,Alabama,US,30.727750,-87.722071,...,65,65,65,66,66,67,67,67,67,69
2,84001005,US,USA,840,1005.0,Barbour,Alabama,US,31.868263,-85.387129,...,9,9,9,9,9,9,9,9,9,9


In [15]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/orig/jh_hist_uscounties_deaths_orig.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=False)

#### 1B: Wrangle the data into more usable format

For the JH live global data, it will be useful to group / subtotal by Country, by US State, and by US County.  

For the historic time series data, these need to be rearranged so that the date is in 
datetime format and pivoted so the date is the index of the DataFrame.

In [16]:
jh_live_global[jh_live_global['Country_Region']=='US'].sort_values('Province_State').head(3)

,FIPS,Admin2,Province_State,Country_Region,Last_Update,Lat,Long_,Confirmed,Deaths,Recovered,Active,Combined_Key,Incidence_Rate,Case-Fatality_Ratio
633,1001.0,Autauga,Alabama,US,2020-10-21 04:24:14,32.539527,-86.644082,2010,29,0,1981.0,"Autauga, Alabama, US",3597.701767,1.442786
670,1075.0,Lamar,Alabama,US,2020-10-21 04:24:14,33.779950,-88.096680,456,5,0,451.0,"Lamar, Alabama, US",3303.151032,1.096491
671,1077.0,Lauderdale,Alabama,US,2020-10-21 04:24:14,34.901719,-87.656247,2087,40,0,2047.0,"Lauderdale, Alabama, US",2250.644351,1.916627


##### 1B-1: Pivot Table on the JH live global data, grouped / subtotalled by **Country**

In [17]:
jh_live_countries = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Confirmed', 'Deaths', 'Recovered'], aggfunc=np.sum)

In [18]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Country_Region,,,
US,8273296,221052,3295148
India,7597063,115197,6733328
Brazil,5273954,154837,4526393


In [19]:
jh_live_countries['Active'] = jh_live_countries['Confirmed'] - jh_live_countries['Deaths'] - jh_live_countries['Recovered']

In [20]:
jh_live_countries['Last_Update'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Last_Update'], aggfunc=np.max)
jh_live_countries['Lat'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Lat'], aggfunc=np.mean)
jh_live_countries['Long'] = pd.pivot_table(jh_live_global, index=['Country_Region'], 
                                   values=['Long_'], aggfunc=np.mean)

In [21]:
jh_live_countries.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Country_Region,,,,,,,
US,8273296,221052,3295148,4757096,2020-10-21 04:24:14,37.936303,-91.379001
India,7597063,115197,6733328,748538,2020-10-21 04:24:14,23.088275,81.806127
Brazil,5273954,154837,4526393,592724,2020-10-21 04:24:14,-12.669522,-48.480493
Russia,1422775,24473,1080461,317841,2020-10-21 04:24:14,54.546312,62.120860
Argentina,1018999,27100,829647,162252,2020-10-21 04:24:14,-38.416100,-63.616700


##### 1B-2: Pivot Table on the JH live global data, grouped / subtotalled by **US State**

In [22]:
jh_live_usstates = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [23]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(3)

,Confirmed,Deaths,Recovered
Province_State,,,
California,883746,17055,0
Texas,862277,17558,0
Florida,760389,16105,0


In [24]:
jh_live_usstates['Active'] = jh_live_usstates['Confirmed'] - jh_live_usstates['Deaths'] - jh_live_usstates['Recovered']

In [25]:
jh_live_usstates['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_usstates['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_usstates['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State'], values=['Long_'], 
                                    aggfunc=np.mean)

In [26]:
jh_live_usstates.sort_values('Confirmed', ascending=False).head(5)

,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long
Province_State,,,,,,,
California,883746,17055,0,866691,2020-10-21 04:24:14,37.843962,-120.728594
Texas,862277,17558,0,844719,2020-10-21 04:24:14,31.660643,-98.653069
Florida,760389,16105,0,744284,2020-10-21 04:24:14,28.940755,-82.700744
New York,486480,33366,0,453114,2020-10-21 04:24:14,42.544151,-75.474183
Illinois,354457,9537,0,344920,2020-10-21 04:24:14,39.843618,-89.178330


##### 1B-3: Pivot Table on the JH live global data, grouped / subtotalled by **US County**

In [27]:
jh_live_uscounties = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Confirmed', 'Deaths', 'Recovered'], 
                                    aggfunc=np.sum)

In [28]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered
Province_State,Admin2,,,
California,Los Angeles,290009,6912,0
Florida,Miami-Dade,179286,3561,0
Illinois,Cook,166457,5369,0
Texas,Harris,155192,2735,0
Arizona,Maricopa,150881,3519,0


In [29]:
jh_live_uscounties['Active'] = jh_live_uscounties['Confirmed'] - jh_live_uscounties['Deaths'] - jh_live_uscounties['Recovered']


In [30]:
jh_live_uscounties['Last_Update'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Last_Update'], 
                                    aggfunc=np.max)
jh_live_uscounties['Lat'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Lat'], 
                                    aggfunc=np.mean)
jh_live_uscounties['Long'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['Long_'], 
                                    aggfunc=np.mean)
jh_live_uscounties['FIPS'] = pd.pivot_table(jh_live_global[jh_live_global['Country_Region']=='US'], 
                                    index=['Province_State', 'Admin2'], values=['FIPS'], 
                                    aggfunc=np.max)

In [31]:
jh_live_uscounties.sort_values('Confirmed', ascending=False).head(5)

,,Confirmed,Deaths,Recovered,Active,Last_Update,Lat,Long,FIPS
Province_State,Admin2,,,,,,,,
California,Los Angeles,290009,6912,0,283097,2020-10-21 04:24:14,34.308284,-118.228241,6037.0
Florida,Miami-Dade,179286,3561,0,175725,2020-10-21 04:24:14,25.611236,-80.551706,12086.0
Illinois,Cook,166457,5369,0,161088,2020-10-21 04:24:14,41.841448,-87.816588,17031.0
Texas,Harris,155192,2735,0,152457,2020-10-21 04:24:14,29.858649,-95.393395,48201.0
Arizona,Maricopa,150881,3519,0,147362,2020-10-21 04:24:14,33.348359,-112.491815,4013.0


##### 1B-4: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [32]:
# First, isolate the date columns of the JH historic time series data to be used in the following 3 Pivot Tables
date_columns = jh_hist_global_cases.iloc[:, 4:].columns

In [33]:
jh_hist_countries_cases = pd.pivot_table(jh_hist_global_cases, index=['Country/Region'], 
                                            values=date_columns, aggfunc=np.sum)
jh_hist_countries_cases = jh_hist_countries_cases.transpose()
jh_hist_countries_cases.index = pd.to_datetime(jh_hist_countries_cases.index)
jh_hist_countries_cases = jh_hist_countries_cases.sort_index()

In [34]:
jh_hist_countries_cases.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-18,40200,17055,54402,3377,7622,119,989680,64694,27399,64806,...,725292,2531,63124,86636,1134,47135,10,2056,15853,8147
2020-10-19,40287,17350,54616,3623,7829,119,1002662,65460,27405,65927,...,744122,2560,63523,86636,1140,47616,10,2056,15897,8159
2020-10-20,40357,17651,54829,3623,8049,119,1018999,66694,27443,67451,...,765487,2623,63831,87644,1141,48129,10,2057,15982,8187


##### 1B-5: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **Country**

In [35]:
jh_hist_countries_deaths = pd.pivot_table(jh_hist_global_deaths, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_deaths = jh_hist_countries_deaths.transpose()
jh_hist_countries_deaths.index = pd.to_datetime(jh_hist_countries_deaths.index)
jh_hist_countries_deaths = jh_hist_countries_deaths.sort_index()

In [36]:
jh_hist_countries_deaths.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-18,1492,451,1856,59,247,3,26267,1081,905,893,...,43736,51,525,736,35,408,1,597,346,231
2020-10-19,1497,454,1865,62,248,3,26716,1091,905,904,...,43816,51,531,736,35,413,1,597,346,232
2020-10-20,1499,458,1873,62,251,3,27100,1101,905,914,...,44057,52,534,747,35,421,1,597,346,233


##### 1B-6: Pivot Table on the JH historic **recoveries**, grouped / subtotalled by **Country**

In [37]:
jh_hist_countries_recovered = pd.pivot_table(jh_hist_global_recovered, index=['Country/Region'], values=date_columns, aggfunc=np.sum)
jh_hist_countries_recovered = jh_hist_countries_recovered.transpose()
jh_hist_countries_recovered.index = pd.to_datetime(jh_hist_countries_recovered.index)
jh_hist_countries_recovered = jh_hist_countries_recovered.sort_index()

In [38]:
jh_hist_countries_recovered.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-18,33614,10071,38088,2057,3030,101,803965,48104,25108,49561,...,2589,2105,60080,79694,1031,40498,8,1338,15005,7678
2020-10-19,33760,10167,38215,2273,3031,101,803965,48208,25112,50359,...,2613,2121,60604,79694,1046,40861,8,1338,15031,7683
2020-10-20,33790,10225,38346,2273,3037,101,829647,48734,25125,51407,...,2626,2142,60910,80316,1046,41455,8,1338,15038,7692


##### 1B-7: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [39]:
jh_hist_uscounties_cases.sort_values('Province_State').head()

,UID,iso2,iso3,code3,FIPS,Admin2,Province_State,Country_Region,Lat,Long_,...,10/11/20,10/12/20,10/13/20,10/14/20,10/15/20,10/16/20,10/17/20,10/18/20,10/19/20,10/20/20
0,84001001,US,USA,840,1001.0,Autauga,Alabama,US,32.539527,-86.644082,...,1905,1911,1924,1928,1949,1966,1983,1989,1999,2010
37,84001075,US,USA,840,1075.0,Lamar,Alabama,US,33.779950,-88.096680,...,420,423,424,426,429,441,446,455,457,456
38,84001077,US,USA,840,1077.0,Lauderdale,Alabama,US,34.901719,-87.656247,...,1918,1935,1944,1950,1974,1994,2029,2043,2063,2087
39,84001079,US,USA,840,1079.0,Lawrence,Alabama,US,34.520415,-87.310695,...,716,729,741,748,759,772,782,788,792,800
40,84001081,US,USA,840,1081.0,Lee,Alabama,US,32.601549,-85.351322,...,6292,6325,6335,6346,6367,6400,6421,6430,6439,6449


In [40]:
date_columns = jh_hist_uscounties_cases.iloc[:,11:].columns

In [41]:
jh_hist_usstates_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_cases = jh_hist_usstates_cases.transpose()
jh_hist_usstates_cases.index = pd.to_datetime(jh_hist_usstates_cases.index)
jh_hist_usstates_cases = jh_hist_usstates_cases.sort_index()

In [42]:
jh_hist_usstates_cases.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-18,172626,10980,0,231149,99066,875692,85279,62830,23093,49,...,228744,854006,94394,1937,1335,165676,98201,20081,166186,9025
2020-10-19,173485,11182,0,231897,99597,879645,86351,64021,23196,49,...,232061,856948,95562,1946,1335,166360,98661,20293,173891,9311
2020-10-20,174528,11391,0,232937,100441,883746,87559,64455,23325,49,...,233569,862277,96643,1956,1337,167270,99150,20520,178482,9526


##### 1B-8: Pivot Table on the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [43]:
jh_hist_uscounties_cases = pd.pivot_table(jh_hist_uscounties_cases, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.transpose()
jh_hist_uscounties_cases.index = pd.to_datetime(jh_hist_uscounties_cases.index)
jh_hist_uscounties_cases = jh_hist_uscounties_cases.sort_index()

In [44]:
jh_hist_uscounties_cases.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-10-18        1989    6369     981  785   1827     634    996    4084   
2020-10-19        1999    6375     981  789   1838     635    998    4102   
2020-10-20        2010    6405     988  791   1848     635    998    4127   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-18         1266      714  ...     440     82      497      142   
2020-10-19         1273      715  ...     443     86      517      145   
2020-10-20         1301      722  ...     462     98      539      149   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-18            415   700   401          0      134     93  
2020-10-19            422   711   403          0      135     99  
2020-10-20            428   716   404          0      139    101  

[3 rows x 3334 columns]

##### 1B-9: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US State**

In [45]:
jh_hist_usstates_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.transpose()
jh_hist_usstates_deaths.index = pd.to_datetime(jh_hist_usstates_deaths.index)
jh_hist_usstates_deaths = jh_hist_usstates_deaths.sort_index()

In [46]:
jh_hist_usstates_deaths.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-18,2788,67,0,5827,1704,16979,2176,4542,665,0,...,2909,17465,543,58,21,3430,2239,402,1574,57
2020-10-19,2789,67,0,5830,1714,16982,2180,4554,666,0,...,2922,17481,546,58,21,3454,2258,402,1600,57
2020-10-20,2805,67,0,5837,1728,17055,2182,4559,668,0,...,2952,17558,551,58,21,3482,2282,411,1633,61


##### 1B-10: Pivot Table on the JH historic **deaths**, grouped / subtotalled by **US County**

In [47]:
jh_hist_uscounties_deaths = pd.pivot_table(jh_hist_uscounties_deaths, index=['Province_State', 'Admin2'], 
                                           values=date_columns, aggfunc=np.sum)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.transpose()
jh_hist_uscounties_deaths.index = pd.to_datetime(jh_hist_uscounties_deaths.index)
jh_hist_uscounties_deaths = jh_hist_uscounties_deaths.sort_index()

In [48]:
jh_hist_uscounties_deaths.tail(3)

Province_State Alabama                                                     \
Admin2         Autauga Baldwin Barbour Bibb Blount Bullock Butler Calhoun   
2020-10-18          28      67       9   13     23      17     40      59   
2020-10-19          28      67       9   13     23      17     40      59   
2020-10-20          29      69       9   14     24      17     40      59   

Province_State                    ... Wyoming                           \
Admin2         Chambers Cherokee  ...    Park Platte Sheridan Sublette   
2020-10-18           43       14  ...       2      1        4        1   
2020-10-19           43       14  ...       2      1        4        1   
2020-10-20           43       14  ...       3      1        4        1   

Province_State                                                    
Admin2         Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-18              2     1     2          0        7      0  
2020-10-19              2     1     2          0        7      0  
2020-10-20              2     1     2          0        7      0  

[3 rows x 3334 columns]

#### 1C: Export the data

##### 1C-1: Export the JH live global data, grouped / subtotalled by **Country**

In [49]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_countries.csv')
jh_live_countries.to_csv(filename, index=True)

##### 1C-2: Export the JH live global data, grouped / subtotalled by **US State**

In [50]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_usstates.csv')
jh_live_usstates.to_csv(filename, index=True)

##### 1C-3: Export the JH live global data, grouped / subtotalled by **US County**

In [51]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_live_uscounties.csv')
jh_live_uscounties.to_csv(filename, index=True)

##### 1C-4: Export the JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [52]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases.csv')
jh_hist_countries_cases.to_csv(filename, index=True)

##### 1C-5: Export the JH historic **deaths**, grouped / subtotalled by **Country**

In [53]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths.csv')
jh_hist_countries_deaths.to_csv(filename, index=True)

##### 1C-6: Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [54]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered.csv')
jh_hist_countries_recovered.to_csv(filename, index=True)

##### 1C-7: Export the JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [55]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases.csv')
jh_hist_usstates_cases.to_csv(filename, index=True)

##### 1C-8: Export the JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [56]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases.csv')
jh_hist_uscounties_cases.to_csv(filename, index=True)

##### 1C-9: Export the JH historic **deaths**, grouped / subtotalled by **US State**

In [57]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths.csv')
jh_hist_usstates_deaths.to_csv(filename, index=True)

##### 1C-10: Export the JH historic **deaths**, grouped / subtotalled by **US County**

In [58]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths.csv')
jh_hist_uscounties_deaths.to_csv(filename, index=True)

#### 1D: Calculate daily % change for each historic dataset (match numbering sequence, export as well)

##### 1D-4: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [59]:
jh_hist_countries_cases_pct = jh_hist_countries_cases.pct_change()

In [60]:
jh_hist_countries_cases_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-18,0.001470,0.016752,0.003671,0.000000,0.021442,0.0,0.010786,0.026889,0.000329,0.026483,...,0.023993,0.011995,0.005015,0.004021,0.007105,0.008322,0.0,0.000487,0.004053,0.004562
2020-10-19,0.002164,0.017297,0.003934,0.072846,0.027158,0.0,0.013117,0.011840,0.000219,0.017298,...,0.025962,0.011458,0.006321,0.000000,0.005291,0.010205,0.0,0.000000,0.002775,0.001473
2020-10-20,0.001738,0.017349,0.003900,0.000000,0.028101,0.0,0.016294,0.018851,0.001387,0.023116,...,0.028712,0.024609,0.004849,0.011635,0.000877,0.010774,0.0,0.000486,0.005347,0.003432


In [61]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct.csv')
jh_hist_countries_cases_pct.to_csv(filename, index=True)

##### 1D-5: Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [62]:
jh_hist_countries_deaths_pct = jh_hist_countries_deaths.pct_change()

In [63]:
jh_hist_countries_deaths_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-18,0.002688,0.006696,0.005417,0.000000,0.024896,0.0,0.006129,0.013121,0.001106,0.004499,...,0.001534,0.000000,0.005747,0.006840,0.0,0.014925,0.0,0.001678,0.0,0.000000
2020-10-19,0.003351,0.006652,0.004849,0.050847,0.004049,0.0,0.017094,0.009251,0.000000,0.012318,...,0.001829,0.000000,0.011429,0.000000,0.0,0.012255,0.0,0.000000,0.0,0.004329
2020-10-20,0.001336,0.008811,0.004290,0.000000,0.012097,0.0,0.014373,0.009166,0.000000,0.011062,...,0.005500,0.019608,0.005650,0.014946,0.0,0.019370,0.0,0.000000,0.0,0.004310


In [64]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct.csv')
jh_hist_countries_deaths_pct.to_csv(filename, index=True)

##### 1D-6: Daily % change Export the JH historic **recoveries**, grouped / subtotalled by **Country**

In [65]:
jh_hist_countries_recovered_pct = jh_hist_countries_recovered.pct_change()

In [66]:
jh_hist_countries_recovered_pct.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-18,0.001579,0.006999,0.003081,0.000000,0.002647,0.0,0.016167,0.003735,0.000398,0.016198,...,0.006610,0.025828,0.005422,0.010742,0.000000,0.008366,0.0,0.001497,0.005225,0.000652
2020-10-19,0.004343,0.009532,0.003334,0.105007,0.000330,0.0,0.000000,0.002162,0.000159,0.016101,...,0.009270,0.007601,0.008722,0.000000,0.014549,0.008963,0.0,0.000000,0.001733,0.000651
2020-10-20,0.000889,0.005705,0.003428,0.000000,0.001980,0.0,0.031944,0.010911,0.000518,0.020811,...,0.004975,0.009901,0.005049,0.007805,0.000000,0.014537,0.0,0.000000,0.000466,0.001171


In [67]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct.csv')
jh_hist_countries_recovered_pct.to_csv(filename, index=True)

##### 1D-7: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [68]:
jh_hist_usstates_cases_pct = jh_hist_usstates_cases.pct_change()

In [69]:
jh_hist_usstates_cases_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-18,0.005616,0.020446,NaN,0.002994,0.006543,0.002379,0.011062,0.000000,0.006582,0.0,...,0.011519,0.006558,0.011758,0.005711,0.000000,0.005346,0.005426,0.013936,0.000000,0.023707
2020-10-19,0.004976,0.018397,NaN,0.003236,0.005360,0.004514,0.012571,0.018956,0.004460,0.0,...,0.014501,0.003445,0.012374,0.004646,0.000000,0.004129,0.004684,0.010557,0.046364,0.031690
2020-10-20,0.006012,0.018691,NaN,0.004485,0.008474,0.004662,0.013989,0.006779,0.005561,0.0,...,0.006498,0.006219,0.011312,0.005139,0.001498,0.005470,0.004956,0.011186,0.026402,0.023091


In [70]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct.csv')
jh_hist_usstates_cases_pct.to_csv(filename, index=True)

##### 1D-8: Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [71]:
jh_hist_uscounties_cases_pct = jh_hist_uscounties_cases.pct_change()

In [72]:
jh_hist_uscounties_cases_pct.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-10-18      0.003026  0.002992  0.004094  0.012903  0.011068  0.000000   
2020-10-19      0.005028  0.000942  0.000000  0.005096  0.006021  0.001577   
2020-10-20      0.005503  0.004706  0.007136  0.002535  0.005441  0.000000   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-10-18      0.006061  0.004427  0.000000  0.001403  ...  0.062802   
2020-10-19      0.002008  0.004407  0.005529  0.001401  ...  0.006818   
2020-10-20      0.000000  0.006095  0.021995  0.009790  ...  0.042889   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-10-18      0.012346  0.008114  0.000000   0.009732  0.011561  0.000000   
2020-10-19      0.048780  0.040241  0.021127   0.016867  0.015714  0.004988   
2020-10-20      0.139535  0.042553  0.027586   0.014218  0.007032  0.002481   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-10-18            NaN  0.007519  0.056818  
2020-10-19            NaN  0.007463  0.064516  
2020-10-20            NaN  0.029630  0.020202  

[3 rows x 3334 columns]

In [73]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct.csv')
jh_hist_uscounties_cases_pct.to_csv(filename, index=True)

##### 1D-9: Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [74]:
jh_hist_usstates_deaths_pct = jh_hist_usstates_deaths.pct_change()

In [75]:
jh_hist_usstates_deaths_pct.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-18,0.000000,0.0,NaN,0.000515,0.011876,0.001356,0.000000,0.000000,0.001506,NaN,...,0.002067,0.001433,0.005556,0.0,0.0,0.003217,0.000000,0.002494,0.000000,0.000000
2020-10-19,0.000359,0.0,NaN,0.000515,0.005869,0.000177,0.001838,0.002642,0.001504,NaN,...,0.004469,0.000916,0.005525,0.0,0.0,0.006997,0.008486,0.000000,0.016518,0.000000
2020-10-20,0.005737,0.0,NaN,0.001201,0.008168,0.004299,0.000917,0.001098,0.003003,NaN,...,0.010267,0.004405,0.009158,0.0,0.0,0.008107,0.010629,0.022388,0.020625,0.070175


In [76]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct.csv')
jh_hist_usstates_deaths_pct.to_csv(filename, index=True)

##### 1D-10: Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [77]:
jh_hist_uscounties_deaths_pct = jh_hist_uscounties_deaths.pct_change()

In [78]:
jh_hist_uscounties_deaths_pct.tail(3)

Province_State   Alabama                                                       \
Admin2           Autauga   Baldwin Barbour      Bibb    Blount Bullock Butler   
2020-10-18      0.000000  0.000000     0.0  0.000000  0.000000     0.0    0.0   
2020-10-19      0.000000  0.000000     0.0  0.000000  0.000000     0.0    0.0   
2020-10-20      0.035714  0.029851     0.0  0.076923  0.043478     0.0    0.0   

Province_State                            ... Wyoming                  \
Admin2         Calhoun Chambers Cherokee  ...    Park Platte Sheridan   
2020-10-18         0.0      0.0      0.0  ...     0.0    0.0      0.0   
2020-10-19         0.0      0.0      0.0  ...     0.0    0.0      0.0   
2020-10-20         0.0      0.0      0.0  ...     0.5    0.0      0.0   

Province_State                                                             
Admin2         Sublette Sweetwater Teton Uinta Unassigned Washakie Weston  
2020-10-18          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-10-19          0.0        0.0   0.0   0.0        NaN      0.0    NaN  
2020-10-20          0.0        0.0   0.0   0.0        NaN      0.0    NaN  

[3 rows x 3334 columns]

In [79]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct.csv')
jh_hist_uscounties_deaths_pct.to_csv(filename, index=True)

#### 1E: Calculate exponential weighted moving average (3 day) for each historic dataset (match numbering sequence, export as well)

This is useful to see which countries / states / counties are becoming hot spots or which are cooling down.  
Exponential weighted moving averages (EWMA) blend the most recent x days, using a heavier weight for more recent data.  
https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html  
Using the EWMA gives the best balance of minimizing erratic daily changes by using a moving avg, while giving more credence to recent data vs a simple moving avg.  

##### 1E-4: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **Country**

In [80]:
jh_hist_countries_cases_pct_ewma = jh_hist_countries_cases_pct.ewm(span=3).mean()

In [81]:
jh_hist_countries_cases_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-18,0.001458,0.016506,0.003711,0.010306,0.025103,0.015923,0.013147,0.025149,0.000389,0.025733,...,0.024314,0.014657,0.004656,0.004793,0.004427,0.007979,1.671814e-37,0.000376,0.004444,0.003272
2020-10-19,0.001811,0.016901,0.003822,0.041576,0.026130,0.007961,0.013132,0.018494,0.000304,0.021515,...,0.025138,0.013057,0.005489,0.002396,0.004859,0.009092,8.359071e-38,0.000188,0.003610,0.002373
2020-10-20,0.001774,0.017125,0.003861,0.020788,0.027116,0.003981,0.014713,0.018673,0.000845,0.022316,...,0.026925,0.018833,0.005169,0.007016,0.002868,0.009933,4.179536e-38,0.000337,0.004478,0.002902


In [82]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_cases_pct_ewma.csv')
jh_hist_countries_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-5: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **Country**

In [83]:
jh_hist_countries_deaths_pct_ewma = jh_hist_countries_deaths_pct.ewm(span=3).mean()

In [84]:
jh_hist_countries_deaths_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-18,0.002243,0.008718,0.004640,0.001408,0.024522,5.137721e-57,0.010648,0.011170,0.000651,0.005882,...,0.002394,0.000199,0.005162,0.007342,5.651697e-16,0.009618,0.0,0.000859,0.000376,0.000145
2020-10-19,0.002797,0.007685,0.004744,0.026128,0.014285,2.568861e-57,0.013871,0.010210,0.000326,0.009100,...,0.002111,0.000099,0.008295,0.003671,2.825848e-16,0.010936,0.0,0.000430,0.000188,0.002237
2020-10-20,0.002067,0.008248,0.004517,0.013064,0.013191,1.284430e-57,0.014122,0.009688,0.000163,0.010081,...,0.003806,0.009854,0.006972,0.009308,1.412924e-16,0.015153,0.0,0.000215,0.000094,0.003274


In [85]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_deaths_pct_ewma.csv')
jh_hist_countries_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-6: EWMA of Daily % change Export the JH historic **recovered**, grouped / subtotalled by **Country**

In [86]:
jh_hist_countries_recovered_pct_ewma = jh_hist_countries_recovered_pct.ewm(span=3).mean()

In [87]:
jh_hist_countries_recovered_pct_ewma.tail(3)

Country/Region,Afghanistan,Albania,Algeria,Andorra,Angola,Antigua and Barbuda,Argentina,Armenia,Australia,Austria,...,United Kingdom,Uruguay,Uzbekistan,Venezuela,Vietnam,West Bank and Gaza,Western Sahara,Yemen,Zambia,Zimbabwe
2020-10-18,0.001699,0.007017,0.003335,0.004771,0.009218,0.003470,0.016362,0.005996,0.000446,0.018817,...,0.004834,0.016443,0.004677,0.009621,0.000313,0.007997,9.795786e-40,0.001501,0.003619,0.002039
2020-10-19,0.003021,0.008275,0.003335,0.054889,0.004774,0.001735,0.008181,0.004079,0.000303,0.017459,...,0.007052,0.012022,0.006700,0.004810,0.007431,0.008480,4.897893e-40,0.000751,0.002676,0.001345
2020-10-20,0.001955,0.006990,0.003381,0.027445,0.003377,0.000867,0.020063,0.007495,0.000410,0.019135,...,0.006014,0.010962,0.005874,0.006308,0.003715,0.011509,2.448947e-40,0.000375,0.001571,0.001258


In [88]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_countries_recovered_pct_ewma.csv')
jh_hist_countries_recovered_pct_ewma.to_csv(filename, index=True)

##### 1E-7: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US State**

In [89]:
jh_hist_usstates_cases_pct_ewma = jh_hist_usstates_cases_pct.ewm(span=3).mean()

In [90]:
jh_hist_usstates_cases_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-18,0.006366,0.019929,NaN,0.003478,0.008284,0.002783,0.012462,0.002133,0.007142,3.232534e-66,...,0.010246,0.006383,0.013435,0.005754,0.001213,0.006099,0.006639,0.014767,0.005660,0.023579
2020-10-19,0.005671,0.019163,NaN,0.003357,0.006822,0.003649,0.012516,0.010545,0.005801,1.616267e-66,...,0.012373,0.004914,0.012904,0.005200,0.000607,0.005114,0.005662,0.012662,0.026012,0.027634
2020-10-20,0.005842,0.018927,NaN,0.003921,0.007648,0.004155,0.013253,0.008662,0.005681,8.081334e-67,...,0.009436,0.005566,0.012108,0.005169,0.001052,0.005292,0.005309,0.011924,0.026207,0.025363


In [91]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_cases_pct_ewma.csv')
jh_hist_usstates_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-8: EWMA of Daily % change JH historic **confirmed cases**, grouped / subtotalled by **US County**

In [92]:
jh_hist_uscounties_cases_pct_ewma = jh_hist_uscounties_cases_pct.ewm(span=3).mean()

In [93]:
jh_hist_uscounties_cases_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-10-18      0.005688  0.003494  0.005867  0.011065  0.011015  0.001939   
2020-10-19      0.005358  0.002218  0.002934  0.008080  0.008518  0.001758   
2020-10-20      0.005430  0.003462  0.005035  0.005307  0.006979  0.000879   

Province_State                                          ...   Wyoming  \
Admin2            Butler   Calhoun  Chambers  Cherokee  ...      Park   
2020-10-18      0.005097  0.008720  0.001297  0.006916  ...  0.048280   
2020-10-19      0.003552  0.006564  0.003413  0.004158  ...  0.027549   
2020-10-20      0.001776  0.006329  0.012704  0.006974  ...  0.035219   

Province_State                                                               \
Admin2            Platte  Sheridan  Sublette Sweetwater     Teton     Uinta   
2020-10-18      0.018570  0.021422  0.007516   0.007722  0.007409  0.007778   
2020-10-19      0.033675  0.030832  0.014321   0.012295  0.011562  0.006383   
2020-10-20      0.086605  0.036692  0.020954   0.013256  0.009297  0.004432   

Province_State                                 
Admin2         Unassigned  Washakie    Weston  
2020-10-18      -0.999999  0.006185  0.075691  
2020-10-19      -0.999999  0.006824  0.070104  
2020-10-20      -0.999999  0.018227  0.045153  

[3 rows x 3334 columns]

In [94]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_cases_pct_ewma.csv')
jh_hist_uscounties_cases_pct_ewma.to_csv(filename, index=True)

##### 1E-9: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US State**

In [95]:
jh_hist_usstates_deaths_pct_ewma = jh_hist_usstates_deaths_pct.ewm(span=3).mean()

In [96]:
jh_hist_usstates_deaths_pct_ewma.tail(3)

Province_State,Alabama,Alaska,American Samoa,Arizona,Arkansas,California,Colorado,Connecticut,Delaware,Diamond Princess,...,Tennessee,Texas,Utah,Vermont,Virgin Islands,Virginia,Washington,West Virginia,Wisconsin,Wyoming
2020-10-18,0.003203,0.008784,NaN,0.001639,0.011610,0.002561,0.001500,0.000132,0.001950,NaN,...,0.005474,0.002979,0.006769,9.361497e-25,0.003125,0.003667,0.000999,0.004808,0.003433,0.000877
2020-10-19,0.001781,0.004392,NaN,0.001077,0.008739,0.001369,0.001669,0.001387,0.001727,NaN,...,0.004971,0.001947,0.006147,4.680749e-25,0.001563,0.005332,0.004743,0.002404,0.009976,0.000439
2020-10-20,0.003759,0.002196,NaN,0.001139,0.008454,0.002834,0.001293,0.001243,0.002365,NaN,...,0.007619,0.003176,0.007652,2.340374e-25,0.000781,0.006719,0.007686,0.012396,0.015300,0.035307


In [97]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_usstates_deaths_pct_ewma.csv')
jh_hist_usstates_deaths_pct_ewma.to_csv(filename, index=True)

##### 1E-10: EWMA of Daily % change JH historic **deaths**, grouped / subtotalled by **US County**

In [98]:
jh_hist_uscounties_deaths_pct_ewma = jh_hist_uscounties_deaths_pct.ewm(span=3).mean()

In [99]:
jh_hist_uscounties_deaths_pct_ewma.tail(3)

Province_State   Alabama                                                    \
Admin2           Autauga   Baldwin   Barbour      Bibb    Blount   Bullock   
2020-10-18      0.000018  0.002484  0.000140  0.002702  0.013688  0.003914   
2020-10-19      0.000009  0.001242  0.000070  0.001351  0.006844  0.001957   
2020-10-20      0.017862  0.015546  0.000035  0.039137  0.025161  0.000979   

Province_State                                              ...       Wyoming  \
Admin2                Butler   Calhoun  Chambers  Cherokee  ...          Park   
2020-10-18      2.484847e-08  0.007424  0.000006  0.002404  ...  2.328306e-10   
2020-10-19      1.242424e-08  0.003712  0.000003  0.001202  ...  1.164153e-10   
2020-10-20      6.212118e-09  0.001856  0.000001  0.000601  ...  2.500000e-01   

Province_State                                                       \
Admin2         Platte      Sheridan Sublette Sweetwater Teton Uinta   
2020-10-18        0.0  3.492460e-10      0.0        0.0   0.0   0.0   
2020-10-19        0.0  1.746230e-10      0.0        0.0   0.0   0.0   
2020-10-20        0.0  8.731149e-11      0.0        0.0   0.0   0.0   

Province_State                              
Admin2         Unassigned  Washakie Weston  
2020-10-18      -0.533333  0.002604    NaN  
2020-10-19      -0.533333  0.001302    NaN  
2020-10-20      -0.533333  0.000651    NaN  

[3 rows x 3334 columns]

In [100]:
filename = os.path.join(os.getcwd(), 'Datasets/JH/jh_hist_uscounties_deaths_pct_ewma.csv')
jh_hist_uscounties_deaths_pct_ewma.to_csv(filename, index=True)

### Dataset 2: US States - Covid Tracking Project

The Covid Tracking Project stores data in both API (JSON) and CSV formats.  We'll be loading the CSV files into Pandas here.  
https://covidtracking.com/api/  

States current - /api/v1/states/current.csv | [CSV](https://covidtracking.com/api/v1/states/current.csv)  
States historic - /api/v1/states/daily.csv | [CSV](https://covidtracking.com/api/v1/states/daily.csv)  
States info - /api/v1/states/info.csv | [CSV](https://covidtracking.com/api/v1/states/info.csv)   

#### 2A: Pull the Data from CT's API, immediately export as 'orig'

In [101]:
ct_live_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/current.csv")
ct_live_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201020,AK,12432,NaN,534093,NaN,totalTestsViral,546525,38.0,NaN,...,546525,0,0,c0e44ad8d86fcc1e637484d963ee4b2a44cca93c,0,0,0,0,0,NaN
1,20201020,AL,174528,21512.0,1112559,NaN,totalTestsViral,1265575,846.0,19081.0,...,1287087,16,0,1fbf9e94e20368495944d9b840b76b5d3299a657,0,0,0,0,0,NaN
2,20201020,AR,100441,6023.0,1137234,NaN,totalTestsViral,1231652,627.0,6428.0,...,1237675,14,67,696a543f35e5a8e1498d893e4b20a56bae36154a,0,0,0,0,0,NaN
3,20201020,AS,0,NaN,1616,NaN,totalTestsViral,1616,NaN,NaN,...,1616,0,0,3321ebf66fbc98011373f6dcb01d358a9c68c1ac,0,0,0,0,0,NaN
4,20201020,AZ,232937,5267.0,1419675,NaN,totalTestsPeopleViral,1647345,777.0,20719.0,...,1652612,7,71,4be9a6265b0b8e6d09e2cb6bff66e9c7690990f1,0,0,0,0,0,NaN


In [102]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_live_usstates_orig.csv')
ct_live_usstates.to_csv(filename, index=False)

In [103]:
ct_hist_usstates = pd.read_csv("https://covidtracking.com/api/v1/states/daily.csv")
ct_hist_usstates.head()

,date,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
0,20201020,AK,12432.0,NaN,534093.0,NaN,totalTestsViral,546525.0,38.0,NaN,...,546525,0,0,c0e44ad8d86fcc1e637484d963ee4b2a44cca93c,0,0,0,0,0,NaN
1,20201020,AL,174528.0,21512.0,1112559.0,NaN,totalTestsViral,1265575.0,846.0,19081.0,...,1287087,16,0,1fbf9e94e20368495944d9b840b76b5d3299a657,0,0,0,0,0,NaN
2,20201020,AR,100441.0,6023.0,1137234.0,NaN,totalTestsViral,1231652.0,627.0,6428.0,...,1237675,14,67,696a543f35e5a8e1498d893e4b20a56bae36154a,0,0,0,0,0,NaN
3,20201020,AS,0.0,NaN,1616.0,NaN,totalTestsViral,1616.0,NaN,NaN,...,1616,0,0,3321ebf66fbc98011373f6dcb01d358a9c68c1ac,0,0,0,0,0,NaN
4,20201020,AZ,232937.0,5267.0,1419675.0,NaN,totalTestsPeopleViral,1647345.0,777.0,20719.0,...,1652612,7,71,4be9a6265b0b8e6d09e2cb6bff66e9c7690990f1,0,0,0,0,0,NaN


In [104]:
filename = os.path.join(os.getcwd(), 'Datasets/CT/orig/ct_hist_usstates_orig.csv')
ct_hist_usstates.to_csv(filename, index=False)

#### 2B: Change the dates to datetime format, make them the index for both datasets

In [105]:
ct_live_usstates['date'] = ct_live_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_live_usstates['date'] = pd.to_datetime(ct_live_usstates['date'])
ct_live_usstates.set_index('date', inplace=True)
ct_live_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-10-20,AK,12432,NaN,534093,NaN,totalTestsViral,546525,38.0,NaN,NaN,...,546525,0,0,c0e44ad8d86fcc1e637484d963ee4b2a44cca93c,0,0,0,0,0,NaN
2020-10-20,AL,174528,21512.0,1112559,NaN,totalTestsViral,1265575,846.0,19081.0,NaN,...,1287087,16,0,1fbf9e94e20368495944d9b840b76b5d3299a657,0,0,0,0,0,NaN
2020-10-20,AR,100441,6023.0,1137234,NaN,totalTestsViral,1231652,627.0,6428.0,257.0,...,1237675,14,67,696a543f35e5a8e1498d893e4b20a56bae36154a,0,0,0,0,0,NaN
2020-10-20,AS,0,NaN,1616,NaN,totalTestsViral,1616,NaN,NaN,NaN,...,1616,0,0,3321ebf66fbc98011373f6dcb01d358a9c68c1ac,0,0,0,0,0,NaN
2020-10-20,AZ,232937,5267.0,1419675,NaN,totalTestsPeopleViral,1647345,777.0,20719.0,170.0,...,1652612,7,71,4be9a6265b0b8e6d09e2cb6bff66e9c7690990f1,0,0,0,0,0,NaN


In [106]:
ct_hist_usstates['date'] = ct_hist_usstates['date'].map(lambda d : str(d)[:4] + '-' + str(d)[4:6] + '-' + str(d)[6:])
ct_hist_usstates['date'] = pd.to_datetime(ct_hist_usstates['date'])
ct_hist_usstates.set_index('date', inplace=True)
ct_hist_usstates.head()

,state,positive,probableCases,negative,pending,totalTestResultsSource,totalTestResults,hospitalizedCurrently,hospitalizedCumulative,inIcuCurrently,...,posNeg,deathIncrease,hospitalizedIncrease,hash,commercialScore,negativeRegularScore,negativeScore,positiveScore,score,grade
date,,,,,,,,,,,,,,,,,,,,,
2020-10-20,AK,12432.0,NaN,534093.0,NaN,totalTestsViral,546525.0,38.0,NaN,NaN,...,546525,0,0,c0e44ad8d86fcc1e637484d963ee4b2a44cca93c,0,0,0,0,0,NaN
2020-10-20,AL,174528.0,21512.0,1112559.0,NaN,totalTestsViral,1265575.0,846.0,19081.0,NaN,...,1287087,16,0,1fbf9e94e20368495944d9b840b76b5d3299a657,0,0,0,0,0,NaN
2020-10-20,AR,100441.0,6023.0,1137234.0,NaN,totalTestsViral,1231652.0,627.0,6428.0,257.0,...,1237675,14,67,696a543f35e5a8e1498d893e4b20a56bae36154a,0,0,0,0,0,NaN
2020-10-20,AS,0.0,NaN,1616.0,NaN,totalTestsViral,1616.0,NaN,NaN,NaN,...,1616,0,0,3321ebf66fbc98011373f6dcb01d358a9c68c1ac,0,0,0,0,0,NaN
2020-10-20,AZ,232937.0,5267.0,1419675.0,NaN,totalTestsPeopleViral,1647345.0,777.0,20719.0,170.0,...,1652612,7,71,4be9a6265b0b8e6d09e2cb6bff66e9c7690990f1,0,0,0,0,0,NaN
